In [2]:
print('hi')

[1] "hi"


In [4]:
print('trial')

[1] "trial"


In [6]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [7]:
read_data <- read_csv2("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/heart-disease.names")
read_data
                       

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 222 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (1): Publication Request:

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Publication Request:
<chr>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
This file describes the contents of the heart-disease directory.
This directory contains 4 databases concerning heart disease diagnosis.
All attributes are numeric-valued. The data was collected from the
four following locations:
1. Cleveland Clinic Foundation (cleveland.data)
"2. Hungarian Institute of Cardiology, Budapest (hungarian.data)"
"3. V.A. Medical Center, Long Beach, CA (long-beach-va.data)"
"4. University Hospital, Zurich, Switzerland (switzerland.data)"
